# Summarize Paper

In [31]:
pip install transformers

     ---------------------------------------- 0.0/44.4 kB ? eta -:--:--
     --------- ------------------------------ 10.2/44.4 kB ? eta -:--:--
     ----------------- -------------------- 20.5/44.4 kB 162.5 kB/s eta 0:00:01
     -------------------------------------- 44.4/44.4 kB 242.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.9 MB 24.8 MB/s eta 0:00:01
   ------- -------------------------------- 1.8/9.9 MB 22.6 MB/s eta 0:00:01
   ------------- -------------------------- 3.2/9.9 MB 25.8 MB/s eta 0:00:01
   ---------------- ----------------------- 4.0/9.9 MB 25.3 MB/s eta 0:00:01
   -------------------- ------------------- 5.1/9.9 MB 25.3 MB/s eta 0:00:01
   ------------------------ --------------- 6.0/9.9 MB 23.9 MB/s eta 0:00:01
   -------------------------- ------------- 6.5/9.9 MB 22.0 MB/s eta 0:00:01
   ------------------------------ --------- 7.5/9.9 MB 22.8 MB/s eta 0:00:01
   -------


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
pip install openai==0.28

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import os
import PyPDF2
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Load the GPT-2 model and tokenizer
def load_gpt2_model():
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    
    # Set the padding token to the EOS token
    tokenizer.pad_token = tokenizer.eos_token

    model = GPT2LMHeadModel.from_pretrained("gpt2")
    return tokenizer, model


# Summarize text using GPT-2
def summarize_text(text, tokenizer, model, max_new_tokens=50, num_return_sequences=1):
    # Tokenize and encode the text input, with padding and attention mask
    inputs = tokenizer("summarize: " + text, return_tensors="pt", max_length=512, truncation=True, padding=True)

    # Generate summary using GPT-2 with attention mask and padding
    outputs = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],  # Add the attention mask
        max_new_tokens=max_new_tokens,  # Limit the number of tokens generated
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id  # Ensure pad_token_id is set
    )

    # Decode the generated output
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary



# Extract text from a PDF
def extract_text_from_pdf(pdf_path):
    pdf_text = ""
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            pdf_text += page.extract_text().replace('\n', ' ')
    return pdf_text

# Main function for summarizing PDF content
def summarize_pdf(pdf_path):
    tokenizer, model = load_gpt2_model()
    pdf_text = extract_text_from_pdf(pdf_path)
    
    # Break the text into chunks for summarization
    chunk_size = 500  # Token limit for GPT-2
    summaries = []
    for i in range(0, len(pdf_text), chunk_size):
        text_chunk = pdf_text[i:i + chunk_size]
        summary = summarize_text(text_chunk, tokenizer, model,max_new_tokens=150)
        summaries.append(summary)
    
    # Combine all summaries into a single text
    final_summary = "\n".join(summaries)
    return final_summary

# Save the summary to a text file
def save_summary(summary, output_file):
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(summary)

if __name__ == "__main__":
    pdf_path = "./lec/paperrml.pdf"  # Path to PDF file
    summary = summarize_pdf(pdf_path)
    
    # Save the summary
    output_file = pdf_path.replace(".pdf", "_summary.txt")
    save_summary(summary, output_file)
    print(f"Summary saved to {output_file}")


Summary saved to ./lec/paperrml_summary.txt


In [42]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cu118



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
import torch
print(torch.__version__)

2.5.0+cpu
